In [99]:
import advent_of_code_2025
lines = advent_of_code_2025.get_lines_py(10,True,1)

In [100]:
lines

['[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}',
 '[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}',
 '[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}']

In [101]:
def parse_line(line):
    line = line.split()
    pat = [0 if x == '.' else 1 for x in line[0][1:-1]]
    n = len(pat)
    def get_button(part):
        b = [0] * n
        for x in part[1:-1].split(','):
            b[int(x)] += 1
        return b
    buttons = [get_button(part) for part in line[1:-1]]
    t = [int(x) for x in line[-1][1:-1].split(',')]
    return pat,buttons,t

parts = [parse_line(line) for line in lines]
parts

[([0, 1, 1, 0],
  [[0, 0, 0, 1],
   [0, 1, 0, 1],
   [0, 0, 1, 0],
   [0, 0, 1, 1],
   [1, 0, 1, 0],
   [1, 1, 0, 0]],
  [3, 5, 4, 7]),
 ([0, 0, 0, 1, 0],
  [[1, 0, 1, 1, 1],
   [0, 0, 1, 1, 0],
   [1, 0, 0, 0, 1],
   [1, 1, 1, 0, 0],
   [0, 1, 1, 1, 1]],
  [7, 5, 12, 7, 2]),
 ([0, 1, 1, 1, 0, 1],
  [[1, 1, 1, 1, 1, 0],
   [1, 0, 0, 1, 1, 0],
   [1, 1, 1, 0, 1, 1],
   [0, 1, 1, 0, 0, 0]],
  [10, 11, 11, 5, 10, 5])]

In [102]:
def min_steps(pat,buttons):
    seen = {tuple([0] * len(pat)): 0}
    for b in buttons:
        nxt = seen.copy()
        for old,step in seen.items():
            t = tuple([(x+y)%2 for x,y in zip(b,old)])
            if t not in nxt:
                nxt[t] = step + 1
            else:
                nxt[t] = min(nxt[t], step + 1)
        seen = nxt
    return seen[tuple(pat)] if tuple(pat) in seen else -1
sum([min_steps(pat,buttons) for pat,buttons,_ in parts])

7

In [103]:
import numpy as np
from scipy.optimize import linprog,milp,LinearConstraint

In [104]:
def min_steps(pat,buttons):
    n = len(buttons)
    A_eq = np.array(buttons).T
    b_eq = np.array(pat)
    c = np.ones(n)
    x = linprog(c,A_eq = A_eq,b_eq=b_eq,integrality=np.ones(n))
    return x
sum(round(min_steps(pat,buttons)['fun']) for _,buttons,pat in parts)

33

In [105]:
def min_steps(pat,buttons):
    n = len(buttons)
    A_eq = np.array(buttons).T
    b_eq = np.array(pat)
    c = np.ones(n)
    x = milp(c,constraints=LinearConstraint(A_eq,lb=b_eq,ub=b_eq,keep_feasible=True),integrality=np.ones_like(n))
    return x
sum(round(min_steps(pat,buttons)['fun']) for _,buttons,pat in parts)

33